In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.stats

import sys
sys.path.append("../../../")

from chiseling.source.protocol.utils import aipw_intercept_pseudo_outcome
from chiseling.dgps.basic_linear_rct import BasicLinearRCT
from chiseling.source.learners.baselearners_general import linreg_learner
from chiseling.source.protocol.IRST import UnitRegistrar, IRST
from chiseling.source.strategies.alpha_spending_strategy import AlphaSpendingStrategy
from chiseling.source.strategies.data_splitting_strategy import DataSplittingStrategy

In [3]:
# Hyperparameters
n = 500
d = 20
s_prog = 20
s_effect = 20
theta_prog = 0
theta_effect = 2
# theta_effect = 1
tau = 0

alpha = 0.05

random_seed = 42

In [4]:
# # Sample data
# sampler = BasicLinearRCT(d, s_prog, s_effect, theta_prog, theta_effect, tau, ipw_transform=True, random_seed=random_seed)
# X, Y = sampler.sample(n)
# # Initialize protocol
# unit_reg = UnitRegistrar(random_seed)
# regX = unit_reg.register_units(X)
# protocol = IRST(regX, Y, alpha=alpha)

In [5]:
# Sample data
sampler = BasicLinearRCT(d, s_prog, s_effect, theta_prog, theta_effect, tau, ipw_transform=False, random_seed=random_seed)
TX, Y = sampler.sample(n)
T, X = TX[:,0], TX[:,1:]
pY = aipw_intercept_pseudo_outcome(T, X, Y, propensities=0.5, random_seed=random_seed)
ipwY = 2 * T * Y - 2 * (1 - T) * Y
# Initialize protocol
unit_reg = UnitRegistrar(random_seed)
regX = unit_reg.register_units(X)
protocol = IRST(regX, ipwY, pY=pY, alpha=alpha)

In [6]:
# Run chiseling
strategy = AlphaSpendingStrategy(protocol=protocol,
                                 test_thresh=0,
                                 learner=linreg_learner,
                                 n_burn_in=0.2,
                                 batch_size=1,
                                 n_min=2,
                                 alpha_min=0,
                                 alpha_spending_fn='uniform',
                                 boundary_strategy='margin',
                                 tiebreak=False,
                                 random_seed=random_seed)

strategy.run_strategy(verbose=True)

METRICS = {'curr_sample_efficiency': 1.0, 'spent_alpha': np.float64(0.0), 'remaining_alpha': np.float64(0.05), 'n_shrink_revealed': 170, 'n_left_in_region': 230, 'region_mass_estimate': np.float64(0.5750000000000012)}
SUMMARY: rejected = True, spent_alpha = 0.010000000000000009, n_left_in_region = 230, region_mass-estimate = 0.5750000000000012


In [7]:
# Run data splitting
datasplit = DataSplittingStrategy(X=X,
                                  Y=Y,
                                  pY=pY,
                                  train_ratio=0.2,
                                  learner=linreg_learner,
                                  alpha=alpha,
                                  test_thresh=0,
                                  random_seed=random_seed)

datasplit.run_strategy()

In [8]:
# Chiseling rejection
protocol.testing_history

,stage_number,mean,variance,sample_size,test_stat,alloc_alpha,remaining_alpha,crit_val,rejection
0,-1,-0.286192,10.968384,500,-1.932286,0.00,0.050000,inf,False
1,0,-0.286192,10.968384,500,-1.932286,0.00,0.050000,inf,False
2,173,0.666968,8.063866,230,3.562025,0.01,0.040404,2.326348,True


In [9]:
# Data split rejection
datasplit.rejected

np.True_

In [10]:
# Chiseling region metrics
sampler.estimate_region_metrics(protocol.rejected_region)

(np.float64(0.7405698986956468),
 np.float64(0.56968),
 np.float64(1.2999752469731196),
 np.float64(0.0038235071901229416),
 np.float64(0.0015657097355512614),
 np.float64(0.005681662988374954))

In [11]:
# Data region metrics
sampler.estimate_region_metrics(datasplit.region)

(np.float64(0.6287403405500746),
 np.float64(0.37102),
 np.float64(1.6946265445261024),
 np.float64(0.0037713702327703586),
 np.float64(0.0015276261309626776),
 np.float64(0.007391917794944131))

In [12]:
# Compare to t-test on points in chiseled region (since we are doing one test)
scipy.stats.ttest_1samp(protocol.pY_masked, popmean=0, alternative='greater')

TtestResult(statistic=np.float64(3.554272800640437), pvalue=np.float64(0.00023014725397082987), df=np.int64(229))

In [13]:
# Get optimal region metrics
sampler.get_optimal_region_metrics()

(np.float64(0.7969571907032398),
 np.float64(0.50077),
 np.float64(1.5914635275740154),
 np.float64(0.003680210458794088),
 np.float64(0.001581136955168653),
 np.float64(0.005362802125466845))

### Test Benchmark module

In [14]:
from chiseling.benchmark.benchmark import Benchmark

In [15]:
settings = {"task_id": 0,
            "dgp": "BasicLinearRCT",
            "n": 500,
            "d": 20,
            "s_prog": 20,
            "s_effect": 20,
            "theta_prog": 0,
            "theta_effect": 0.5,
            "tau": 0,
            "ipw_transform": True,
            "alpha": 0.05,
            "random_seed": 42,
            "strategy": "AlphaSpendingStrategy",
            "test_thresh": 0,
            "learner": "linreg_learner",
            "n_burn_in": 0.8,
            "batch_size": 1,
            "n_min": 2,
            "alpha_min": 0,
            "alpha_spending_fn": "instantaneous",
            "binary": False,
            "substitute_t_test": True,
            "n_sims": 10}

In [16]:
benchmark = Benchmark(settings)

In [17]:
benchmark.simulate_batch()

In [18]:
benchmark.simulation_results_df

,task_id,sim_id,rejected,region_mass,subgroup_mean,subgroup_utility,region_mass_se,subgroup_mean_se,subgroup_utility_se
0,0,0,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
1,0,1,True,0.47822,0.323099,0.154512,0.001580,0.001786,0.000995
2,0,2,True,0.42873,0.359908,0.154303,0.001565,0.001850,0.000973
3,0,3,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
4,0,4,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
5,0,5,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
6,0,6,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
7,0,7,True,0.48374,0.347568,0.168132,0.001580,0.001672,0.000978
8,0,8,True,0.49627,0.286581,0.142221,0.001581,0.001843,0.001021
9,0,9,True,0.47354,0.309438,0.146531,0.001579,0.001851,0.001003


In [19]:
settings = {"task_id": 0,
            "dgp": "BasicLinearRCT",
            "n": 500,
            "d": 20,
            "s_prog": 20,
            "s_effect": 20,
            "theta_prog": 0,
            "theta_effect": 0.5,
            "tau": 0,
            "ipw_transform": True,
            "alpha": 0.05,
            "random_seed": 42,
            "strategy": "DataSplittingStrategy",
            "test_thresh": 0,
            "learner": "linreg_learner",
            "train_ratio": 0.2,
            "binary": False,
            "n_sims": 10}

In [20]:
benchmark = Benchmark(settings)

In [21]:
benchmark.simulate_batch()

In [22]:
benchmark.simulation_results_df

,task_id,sim_id,rejected,region_mass,subgroup_mean,subgroup_utility,region_mass_se,subgroup_mean_se,subgroup_utility_se
0,0,0,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
1,0,1,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
2,0,2,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
3,0,3,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
4,0,4,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
5,0,5,True,0.42219,0.285876,0.120694,0.001562,0.002096,0.000991
6,0,6,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
7,0,7,True,0.40213,0.280406,0.112760,0.001551,0.002179,0.000978
8,0,8,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
9,0,9,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000


In [23]:
settings = {"task_id": 0,
            "dgp": "BasicLinearRCT",
            "n": 500,
            "d": 20,
            "s_prog": 20,
            "s_effect": 20,
            "theta_prog": 0,
            "theta_effect": 0.5,
            "tau": 0,
            "ipw_transform": True,
            "alpha": 0.05,
            "random_seed": 42,
            "strategy": "OracleStrategy",
            "test_thresh": 0,
            "learner": "linreg_learner",
            "train_ratio": 0.2,
            "binary": False,
            "n_sims": 10}

In [24]:
benchmark = Benchmark(settings)

In [25]:
benchmark.simulate_batch()

In [26]:
benchmark.simulation_results_df

,task_id,sim_id,rejected,region_mass,subgroup_mean,subgroup_utility,region_mass_se,subgroup_mean_se,subgroup_utility_se
0,0,0,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
1,0,1,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
2,0,2,True,0.57605,0.229341,0.132112,0.001563,0.001776,0.001084
3,0,3,True,0.42894,0.398204,0.170805,0.001565,0.001688,0.000955
4,0,4,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
5,0,5,False,0.00000,NaN,0.000000,0.000000,NaN,0.000000
6,0,6,True,0.42527,0.373723,0.158933,0.001563,0.001806,0.000965
7,0,7,True,0.44264,0.323398,0.143149,0.001571,0.001921,0.000990
8,0,8,True,0.39841,0.353922,0.141006,0.001548,0.001992,0.000965
9,0,9,True,0.53207,0.286438,0.152405,0.001578,0.001730,0.001026
